# Exploratory Data Analysis
## Book Recommender System Dataset

Tujuan EDA:
- Memahami karakteristik data
- Menemukan pola rating
- Mengidentifikasi masalah data (missing, sparsity, dll)


**Import Library**

In [7]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [8]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [9]:
!ls "/content/drive/MyDrive/Colab Notebooks/BigDataBooks"

books_clean.csv  ratings_clean.csv  user_recommendations      users_clean.csv
books.csv	 ratings.csv	    user_recommendations.csv  users.csv


In [4]:
import pandas as pd

books = pd.read_csv(BASE_PATH + 'books.csv')
users = pd.read_csv(BASE_PATH + 'users.csv')
ratings = pd.read_csv(BASE_PATH + 'ratings.csv')


NameError: name 'BASE_PATH' is not defined

In [10]:
print("Books:", books.shape)
print("Users:", users.shape)
print("Ratings:", ratings.shape)

books.head()


NameError: name 'books' is not defined

In [ ]:
books = pd.read_csv('/content/books.csv')
users = pd.read_csv('/content/users.csv')
ratings = pd.read_csv('/content/ratings.csv')

**Pemeriksaan Awal Dataset Books**

In [ ]:
books.head()
books.info()
books.describe()
books.isnull().sum()

### Kesimpulan Awal Dataset Books

Berdasarkan pemeriksaan awal dataset **books**, diperoleh kesimpulan sebagai berikut:

- Dataset books terdiri dari **271.360 baris dan 9 kolom** yang memuat informasi terkait buku, seperti ISBN, judul buku, penulis, tahun terbit, penerbit, dan URL gambar.
- Sebagian besar kolom bertipe data teks (object), sedangkan kolom tahun publikasi bertipe numerik.
- Terdapat **missing values dalam jumlah sangat kecil**, yaitu pada kolom `Book-Author`, `Publisher`, dan `Image-URL-L`.
- Missing values yang ada relatif tidak signifikan dibandingkan ukuran dataset, namun tetap perlu ditangani pada tahap preprocessing.
- Dataset books layak digunakan sebagai sumber data utama untuk sistem rekomendasi berbasis buku.


**Pemeriksaan Awal Dataset Users**

In [ ]:
users.head()
users.info()
users.describe()
users.isnull().sum()

### Kesimpulan Awal Dataset Users

Dataset **users** terdiri dari **278.858 baris dan 4 kolom** yang menyimpan informasi pengguna berupa ID pengguna, lokasi, dan usia.

- Kolom `Age` memiliki **110.762 nilai kosong**, yang menunjukkan bahwa sebagian besar pengguna tidak mengisi informasi usia.
- Nilai usia juga berpotensi mengandung data tidak valid (usia sangat kecil atau sangat besar).
- Informasi lokasi disimpan dalam bentuk teks dan tidak memiliki missing value.
- Dataset users digunakan sebagai identitas pengguna dalam sistem rekomendasi dan tidak menjadi fitur utama dalam pemodelan awal.


**Pemeriksaan Awal Dataset Ratings**

In [ ]:
ratings.head()
ratings.info()
ratings.describe()
ratings.isnull().sum()

### Kesimpulan Awal Dataset Ratings

Dataset **ratings** terdiri dari **1.149.780 baris dan 4 kolom** yang merepresentasikan interaksi antara pengguna dan buku.

- Setiap baris menunjukkan satu aktivitas pemberian rating oleh seorang pengguna terhadap sebuah buku.
- Rating diberikan dalam skala **0 hingga 10**, di mana nilai 0 menunjukkan implicit feedback (buku dibaca/tidak diberi rating eksplisit).
- Distribusi rating tidak merata, dengan median bernilai 0, yang menandakan banyak pengguna tidak memberikan rating eksplisit.
- Dataset ini merupakan komponen utama dalam pembangunan sistem rekomendasi, khususnya metode collaborative filtering.


### Kesimpulan Pemeriksaan Awal Data

Berdasarkan pemeriksaan awal terhadap dataset books, users, dan ratings, dapat disimpulkan bahwa:

- Ketiga dataset saling terhubung melalui kolom `ISBN` dan `User-ID`.
- Dataset memiliki ukuran besar dan bersifat **sparse**, dengan jumlah interaksi yang jauh lebih sedikit dibandingkan kemungkinan pasangan user–item.
- Terdapat missing values terutama pada dataset users, khususnya pada kolom usia.
- Karakteristik data ini sangat cocok untuk penerapan sistem rekomendasi menggunakan metode **Content-based Recommender System** maupun **Collaborative Filtering**.

Tahap selanjutnya yang diperlukan adalah **penanganan missing values dan preprocessing data** sebelum membangun model rekomendasi.


In [ ]:
print("Missing values - Books")
display(books.isnull().sum())

print("\nMissing values - Users")
display(users.isnull().sum())

print("\nMissing values - Ratings")
display(ratings.isnull().sum())

### Identifikasi Missing Values

Berdasarkan pemeriksaan awal, ditemukan bahwa:
- Dataset **books** memiliki missing values dalam jumlah sangat kecil.
- Dataset **users** memiliki missing values yang signifikan pada kolom usia (Age).
- Dataset **ratings** tidak memiliki missing values.

Oleh karena itu, diperlukan strategi penanganan missing values yang berbeda untuk setiap dataset.

**Data Cleaning Dataset Books**

In [ ]:
Books_clean = books.dropna()
Books_clean.isnull().sum()

### Data Cleaning Dataset Books

Pada dataset books, jumlah missing values sangat kecil dan tidak signifikan dibandingkan total data.

Oleh karena itu, dipilih metode **penghapusan baris (drop)** yang mengandung missing values karena:
- Tidak memengaruhi distribusi data secara keseluruhan
- Menjaga konsistensi informasi buku
- Lebih sederhana dan aman untuk pemodelan

Setelah proses cleaning, dataset books tidak lagi memiliki missing values.


**Data Cleaning Dataset Users**

In [ ]:
users_clean = users.copy()
users_clean['Age'] = users_clean['Age']

### Data Cleaning Dataset Users

Kolom `Age` pada dataset users memiliki jumlah missing values yang cukup besar (lebih dari 40%).

Dalam konteks sistem rekomendasi berbasis rating:
- Informasi usia tidak menjadi fitur utama dalam pemodelan
- Melakukan imputasi dapat menimbulkan bias
- Menghapus baris akan mengurangi jumlah user secara signifikan

Oleh karena itu, kolom `Age` dipertahankan namun **tidak digunakan dalam proses pemodelan recommender system**.


**Data Cleaning Dataset Ratings**

In [ ]:
ratings_clean = ratings.copy()

### Data Cleaning Dataset Ratings

Dataset ratings tidak memiliki missing values sehingga tidak memerlukan penanganan khusus.

Nilai rating 0 dianggap sebagai **implicit feedback**, yang menunjukkan interaksi pengguna dengan buku tanpa penilaian eksplisit.

Nilai ini tetap dipertahankan karena masih relevan dalam sistem rekomendasi, terutama untuk pendekatan collaborative filtering.


In [ ]:
books_clean = books.copy()
users_clean = users.copy()
ratings_clean = ratings.copy()


In [ ]:
# Isi missing value pada Book-Author
books_clean['Book-Author'] = books_clean['Book-Author'].fillna('Unknown')

# Drop baris tanpa judul
books_clean = books_clean.dropna(subset=['Book-Title'])


In [ ]:
# Isi missing age dengan median
users_clean['Age'] = users_clean['Age'].fillna(users_clean['Age'].median())


In [ ]:
# Pastikan rating numerik
ratings_clean['Book-Rating'] = ratings_clean['Book-Rating'].astype(int)


**Validasi Hasil Cleaning**

In [ ]:
print("Missing values setelah cleaning:")
print("Books:", books_clean.isnull().sum().sum())
print("Users:", users_clean.isnull().sum().sum())
print("Ratings:", ratings_clean.isnull().sum().sum())


### Validasi Data Setelah Cleaning

Setelah dilakukan proses data cleaning:
- Dataset books tidak lagi memiliki missing values
- Dataset users masih memiliki missing values pada kolom Age yang tidak digunakan dalam pemodelan
- Dataset ratings tetap utuh tanpa missing values

Dataset siap digunakan untuk tahap eksplorasi lanjutan dan pemodelan sistem rekomendasi.

**SIMPAN DATA HASIL CLEANING**

In [ ]:
books_clean.to_csv('books_clean.csv', index=False)
users_clean.to_csv('users_clean.csv', index=False)
ratings_clean.to_csv('ratings_clean.csv', index=False)

**Distribusi Rating**

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

plt.figure(figsize=(8,5))
sns.countplot(x=ratings_clean['Book-Rating'])
plt.title('Distribusi Rating Buku')
plt.xlabel('Rating')
plt.ylabel('Jumlah')
plt.show()

### Distribusi Rating

Berdasarkan visualisasi distribusi rating, terlihat bahwa:
- Nilai rating **0 mendominasi dataset**, menunjukkan banyak interaksi bersifat implicit feedback.
- Rating eksplisit (1–10) jumlahnya jauh lebih sedikit.
- Distribusi rating tidak merata dan cenderung skewed.

Kondisi ini menunjukkan bahwa dataset bersifat sparse dan cocok untuk sistem rekomendasi.


**Aktivitas User (User Activity)**

In [ ]:
user_activity = ratings_clean.groupby('User-ID').size()

plt.figure(figsize=(8,5))
sns.histplot(user_activity, bins=50)
plt.title('Distribusi Jumlah Rating per User')
plt.xlabel('Jumlah Rating')
plt.ylabel('Jumlah User')
plt.show()

### Aktivitas Pengguna

Distribusi aktivitas pengguna menunjukkan bahwa:
- Sebagian besar pengguna hanya memberikan sedikit rating.
- Hanya sebagian kecil pengguna yang sangat aktif memberikan banyak rating.
- Distribusi bersifat long-tail.

Hal ini merupakan karakteristik umum pada dataset sistem rekomendasi dan menjadi tantangan dalam pemodelan.


**Popularitas Buku (Item Popularity)**

In [ ]:
book_popularity = ratings_clean.groupby('ISBN').size()

plt.figure(figsize=(8,5))
sns.histplot(book_popularity, bins=50)
plt.title('Distribusi Jumlah Rating per Buku')
plt.xlabel('Jumlah Rating')
plt.ylabel('Jumlah Buku')
plt.show()

### Popularitas Buku

Hasil analisis popularitas buku menunjukkan bahwa:
- Sebagian kecil buku memiliki jumlah rating yang sangat tinggi.
- Sebagian besar buku hanya memiliki sedikit atau bahkan satu rating.
- Fenomena ini dikenal sebagai **long-tail distribution**.

Kondisi ini berpotensi menyebabkan masalah cold-start pada item.

**Buku Paling Populer (Top-N)**

In [ ]:
top_books = ratings_clean.groupby('ISBN').size().sort_values(ascending=False).head(10)
top_books
top_books_df = top_books.reset_index(name='rating_count')
top_books_df.merge(books_clean[['ISBN','Book-Title']], on='ISBN', how='left')

### Buku dengan Jumlah Rating Terbanyak

Beberapa buku memiliki jumlah rating yang jauh lebih tinggi dibandingkan buku lainnya.
Buku-buku ini dapat dianggap sebagai buku populer yang sering muncul dalam rekomendasi berbasis popularitas.


**Sparsity Dataset**

In [ ]:
num_users = ratings_clean['User-ID'].nunique()
num_books = ratings_clean['ISBN'].nunique()
num_ratings = len(ratings_clean)

sparsity = 1 - (num_ratings / (num_users * num_books))
sparsity

### Sparsity Dataset

Tingkat sparsity dihitung sebagai perbandingan antara jumlah rating aktual dan kemungkinan pasangan user–buku.

Hasil perhitungan menunjukkan bahwa dataset memiliki tingkat sparsity yang sangat tinggi.
Artinya, sebagian besar pengguna hanya memberikan rating pada sebagian kecil dari seluruh buku yang tersedia.

Kondisi ini menegaskan perlunya penggunaan sistem rekomendasi untuk memprediksi preferensi pengguna.


### Insight Utama dari Exploratory Data Analysis

Berdasarkan Exploratory Data Analysis yang telah dilakukan, diperoleh beberapa temuan penting:

1. Dataset bersifat **sparse** dengan dominasi implicit feedback.
2. Aktivitas pengguna dan popularitas buku mengikuti distribusi long-tail.
3. Banyak buku dan pengguna memiliki sedikit interaksi (cold-start problem).
4. Rating eksplisit relatif sedikit dibandingkan total interaksi.

Berdasarkan karakteristik tersebut, sistem rekomendasi berbasis **Collaborative Filtering** dipilih karena mampu memanfaatkan pola interaksi antar pengguna dan item.
Sebagai pembanding, pendekatan **Content-based** juga dapat digunakan untuk mengatasi cold-start pada item baru.


In [ ]:
ratings_rs = ratings_clean[['User-ID', 'ISBN', 'Book-Rating']]
books_rs = books_clean[['ISBN', 'Book-Title', 'Book-Author']]
users_rs = users_clean[['User-ID']]

### Seleksi Kolom untuk Recommender System

Pada tahap preprocessing, hanya kolom yang relevan untuk sistem rekomendasi yang dipertahankan.
Hal ini bertujuan untuk mengurangi kompleksitas dan fokus pada relasi user–item–rating.


In [ ]:
ratings_rs = ratings_rs[ratings_rs['Book-Rating'] > 0]

### Penanganan Rating Implicit

Nilai rating 0 diinterpretasikan sebagai implicit feedback.
Untuk meningkatkan kualitas rekomendasi berbasis collaborative filtering,
hanya rating eksplisit (rating > 0) yang digunakan pada tahap ini.


In [ ]:
# filter user
user_counts = ratings_rs['User-ID'].value_counts()
valid_users = user_counts[user_counts >= 5].index
ratings_rs = ratings_rs[ratings_rs['User-ID'].isin(valid_users)]

# filter book
book_counts = ratings_rs['ISBN'].value_counts()
valid_books = book_counts[book_counts >= 5].index
ratings_rs = ratings_rs[ratings_rs['ISBN'].isin(valid_books)]


### Filtering User dan Buku dengan Interaksi Rendah

Pengguna dan buku dengan jumlah rating yang sangat sedikit dapat menimbulkan noise.
Oleh karena itu, hanya pengguna dan buku dengan minimal 5 interaksi yang dipertahankan.
Langkah ini membantu meningkatkan stabilitas model rekomendasi.


In [ ]:
ratings_rs['rating_scaled'] = ratings_rs['Book-Rating'] / 2

### Normalisasi Skala Rating

Skala rating dinormalisasi dari rentang 1–10 menjadi 1–5
untuk meningkatkan kestabilan model collaborative filtering.


In [ ]:
from sklearn.preprocessing import LabelEncoder

user_encoder = LabelEncoder()
book_encoder = LabelEncoder()

ratings_rs['user_idx'] = user_encoder.fit_transform(ratings_rs['User-ID'])
ratings_rs['book_idx'] = book_encoder.fit_transform(ratings_rs['ISBN'])


In [ ]:
# Ambil mapping ISBN -> book_idx dari ratings_clean
book_mapping = ratings_clean[['ISBN', 'book_idx']].drop_duplicates()

# Gabungkan ke books_clean
books_clean = books_clean.merge(
    book_mapping,
    on='ISBN',
    how='left'
)

books_clean.head()

### Encoding User dan Buku

Algoritma ALS memerlukan ID pengguna dan item dalam bentuk numerik.
Oleh karena itu, dilakukan encoding pada User-ID dan ISBN menjadi integer index.


In [ ]:
from sklearn.model_selection import train_test_split

train_df, test_df = train_test_split(
    ratings_rs[['user_idx', 'book_idx', 'rating_scaled']],
    test_size=0.2,
    random_state=42
)


### Pembagian Data Train dan Test

Dataset dibagi menjadi data latih dan data uji dengan rasio 80:20
untuk mengevaluasi performa model rekomendasi.


In [ ]:
train_df.to_csv('train_ratings.csv', index=False)
test_df.to_csv('test_ratings.csv', index=False)


### Dataset Siap Digunakan untuk Pemodelan

Data hasil preprocessing disimpan untuk digunakan
pada tahap pemodelan recommender system berbasis PySpark.


In [ ]:
SAVE_PATH = '/content/drive/MyDrive/Colab Notebooks/BigDataBooks/'

books_clean.to_csv(SAVE_PATH + 'books_clean.csv', index=False)
users_clean.to_csv(SAVE_PATH + 'users_clean.csv', index=False)
ratings_clean.to_csv(SAVE_PATH + 'ratings_clean.csv', index=False)
